# Sistemas Operacionais
## Laboratório 02 - Threads e comunicação entre processos

### **Introdução**

Neste laboratório aprenderemos a realizar a comunicação entre processos e a manipulação de threads usando o pacote POSIX e a linguagem C no Linux. O foco desta aula será no aspecto prático do conteúdo, todavia um breve resumo da teoria será exposto, quando necessário, para auxiliar na execução dos exercícios.

Ao final desta aula, o estudante deverá ser capaz de:
* Fazer o estabelecimento de uma área de memória compartilhada;
* Realizar a criação de threads;
* Fazer a manipulação de variáveis via memória compartilhada e threads.

### **Instruções**

Esse laboratório foi feito usando a ferramenta Colab do Google para evitar a necessidade de instalação de programas extras e máquinas virtuais. Para completá-lo, é esperado que o aluno siga os comandos em cada etapa e procure entender o que acontece com o sistema quando determinado código é executado. É recomendado também que o aluno tente introduzir pequenas modificações nos códigos fornecidos para testar seu aprendizado.

Além disso, este material contém 6 exercícios práticos, cada um valendo 0,75 pontos na nota da unidade 1. **O estudante deve escolher 4 deles** para resolver, enviando um breve relatório contendo capturas de tela das saídas obtidas para o SIGAA ao final do exercício. Além disso, os códigos criados também deverão ser entregues. O prazo de submissão da atividade vai até as 23:59 do dia 21/10/2024. **A professora se absterá de dar orientações sobre programação até o prazo final do exercício, todavia o estudante poderá consultar todo o material que achar necessário**.


### **Parte 1 - Comunicação entre processos via compartilhamento de memória**
Como vimos em nossas aulas teóricas, os processos têm espaços de endereçamentos próprios, os quais, por via de regra, não se comunicam entre si. Quando um processo deseja trocar informações com outros processos, ele pode usar dois modelos de comunicação: transmissão de mensagens ou memória compartilhada.

No modelo de comunicação via memória compartilhada, os processos comunicam ao sistema operacional que desejam estabelecer uma região de memória que poderá ser acessada por mais de um processo. Isto é feito por meio das chamadas de sistema *shmget*, que cria a memória compartilhada, e *shmat* que anexa a memória compartilhada ao espaço de endereçamento do processo, retornando um ponteiro de acesso à tal memória.

Ao final da comunicação, os processos podem desanexar a memória por meio da chamada de sistema *shmdt* e removê-la via *shmctl*. Observe a utilização de tais chamadas de sistema no código abaixo, no qual o processo pai lê o que o filho escreveu em um espaço de memória compartilhada.

In [14]:
%%writefile sharedMemory.c
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <unistd.h>
#include <sys/shm.h>
#include <sys/stat.h>
#include <stdlib.h>

int main(int argc, char **argv)
{
  // Criando o segmento de memoria compartilhada com o tamanho de 10 chars
  int segment_id=shmget(IPC_PRIVATE, 10*sizeof(char),S_IRUSR|S_IWUSR);
  // Criando o ponteiro que vai apontar para o endereço de memória compartilhada
  char *sharedMemory;
  pid_t pid=fork(); // Funcao usada para criar um novo processo
  if(pid<0){ // Erro na criacao do processo
    printf("Sou o processo %d e nao consegui criar um novo processo.\n",getpid());
    return 1;
  }else if(pid==0){ // Processo filho
    // Anexando o segmento de memoria compartilhada
    sharedMemory=(char *) shmat(segment_id, NULL,0);
    // Escrevendo na memória compartilhada
    sprintf(sharedMemory, "Olá, processo pai, sou o seu filho e o meu ID é %d.",getpid());
    // Desanexando a memória compartilhada
    shmdt(sharedMemory);
    // Encerrando o processo filho
    exit(0);
  }else{ // Processo pai
    // Fazendo o pai esperar que o filho termine de escrever
    wait(NULL);
    // Salvando o ponteiro que aponta para a memória compartilhada
    sharedMemory=(char *) shmat(segment_id, NULL,0);
    // Imprimindo mensagem deixada pelo processo filho
    printf("%s\n",sharedMemory);
    // Removendo o segmento de memória compartilhada
    shmctl(segment_id, IPC_RMID,NULL);
  }
  return 0;
}


Overwriting sharedMemory.c


In [15]:
!gcc ./sharedMemory.c -o sharedMemory
!./sharedMemory

Olá, processo pai, sou o seu filho e o meu ID é 17442.


**Exercício 1**

Modifique o código acima para fazer com que dois processos filhos alterem uma memória compartilhada contendo um inteiro. Faça o processo pai esperar por seus dois filhos para depois imprimir o valor da variável na tela. Dica: olhe o código do exercício 3 do Laboratório 1.


---



**Exercício 2**

Utilizando comunicação entre processos, crie um código que escreva a Sequência de Fibonacci, com o parâmetro n sendo uma variável global. Tal sequência deve ser gerada por um processo filho, mas deve ser impressa na tela pelo processo pai. Dica: olhe o código do desafio do Laboratório 1.


---



**Exercício 3**

Usando comunicação entre processos, escreva um código que realiza a soma de *n* números naturais, sendo *n* uma variável global. Cada processo deve fazer apenas a soma de dois números e o resultado final deve ser retornado pelo primeiro processo. Por exemplo, o processo 1 deve fazer a soma de 0+1, o processo 2 vai pegar esse resultado e somar com 2, e assim por diante.


---



### **Parte 2 - Threads**
Threads são linhas de execução. Cada processo tem ao menos uma thread, mas pode conter várias, dividindo assim sua execução em várias tarefas menores.

Para criar uma thread, usamos a função *pthread_create*. Tal função retorna um int que indica o status da criação, sendo 0, se ela foi bem sucedida, ou outro valor, se houve algum problema. Além disso, ela tem os seguintes argumentos:
1. Ponteiro que armazena o identificador da thread;
2. Atributos da thread. Caso queira os atributos padrão, use NULL;
3. Ponteiro da função a ser usada para a thread. Tal função tem um único argumento: um ponteiro para void;
4. Um ponteiro para o argumento da função do ponto 3.

Outras funções úteis no gerenciamento das threads são a *pthread_exit*, a qual encerra a thread, e a *pthread_join*, que faz uma thread aguardar o encerramento de outra thread especificada. Veja a utilização dessas funções no exemplo abaixo, que faz *n* threads imprimirem uma mensagem na tela.





In [3]:
|%%writefile helloThreads.c
#include <pthread.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <unistd.h>
#include <sys/shm.h>
#include <sys/stat.h>
#include <stdlib.h>

// Variavel global que define o numero de threads
int numberOfThreads=5;

// Funcao que as threads vao usar para dar um ola
void *printHello(void *tid){
  printf("Olá, eu sou a thread %d\n", (int)(size_t) tid);
  pthread_exit(NULL);
}

int main(int argc, char **argv)
{
  pthread_t threads[numberOfThreads];
  for (int i=0;i<numberOfThreads;i++){
    // Criando uma thread e passando para ela a funcao printHello
    int status=pthread_create(&threads[i],NULL,printHello,(void *)(size_t) i);

    // Se o status nao for 0, houve um erro na criacao da thread
    if(status!=0){
      printf("Erro na criação da thread %d. Código de erro %d.",i, status);
      return 1;
    }

    // Faz a thread principal esperar pela thread filha. Similar a funcao wait.
    pthread_join(threads[i],NULL);
  }
  return 0;
}

Writing helloThreads.c


In [4]:
!gcc ./helloThreads.c -o helloThreads
!./helloThreads

Olá, eu sou a thread 0
Olá, eu sou a thread 1
Olá, eu sou a thread 2
Olá, eu sou a thread 3
Olá, eu sou a thread 4


**Exercício 4**

Comente a linha *pthread_join(threads[i],NULL)* no código acima e veja o que acontece. Qual o motivo da mudança no output? Faça modificações no código para obter novamente a saída original sem necessitar usar um *threads_join*. Dica: threads também podem dormir.



---



Threads de um mesmo processo compartilham do mesmo espaço de endereçamento deste. Desta forma, todas elas têm acesso às variáveis globais do processo, podendo modificá-las sem grandes complicações.

Veja no código abaixo como incrementar uma variável global fazendo uso de threads.

In [13]:
%%writefile multipleThreadsOneVariable.c
#include <pthread.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <unistd.h>
#include <sys/shm.h>
#include <sys/stat.h>
#include <stdlib.h>

// Variavel global que define o numero de threads
int numberOfThreads=5;
// Variável global que define valor
int valor=0;

// Funcao que as threads vao usar para alterar valor
void *alterarValor(void *tid){
  valor=valor+5;
  printf("Olá, eu sou a thread %d. Eu alterei valor para %d\n", (int)(size_t) tid, valor);
  pthread_exit(NULL);
}

int main(int argc, char **argv)
{
  pthread_t threads[numberOfThreads];
  for (int i=0;i<numberOfThreads;i++){
    // Criando uma thread e passando para ela a funcao alterarValor
    int status=pthread_create(&threads[i],NULL,alterarValor,(void *)(size_t) i);

    // Se o status nao for 0, houve um erro na criacao da thread
    if(status!=0){
      printf("Erro na criação da thread %d. Código de erro %d.",i, status);
      return 1;
    }

    // Faz a thread principal esperar pela thread filha. Similar a funcao wait.
    pthread_join(threads[i],NULL);
  }
  printf("Olá, eu sou a thread original. Valor é igual a %d\n", valor);
  return 0;
}

Overwriting multipleThreadsOneVariable.c


In [11]:
!gcc ./multipleThreadsOneVariable.c -o multipleThreadsOneVariable
!./multipleThreadsOneVariable

Olá, eu sou a thread 0. Eu alterei valor para 5
Olá, eu sou a thread 1. Eu alterei valor para 10
Olá, eu sou a thread 2. Eu alterei valor para 15
Olá, eu sou a thread 3. Eu alterei valor para 20
Olá, eu sou a thread 4. Eu alterei valor para 25
Olá, eu sou a thread original. Valor é igual a 25


**Exercício 5**

Refaça o exercício 2 usando threads.


---



**Exercício 6**

Refaça o exercício 3 usando threads.


---



